In [2]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import concatenate, add
from keras.models import Model
from keras.regularizers import l2
import keras.backend as K

from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.callbacks import TensorBoard

import numpy as np 
import json
import os
import cv2
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
from helpers import TrainingMonitor
from helpers import Utils

In [4]:
output_path = "../output/"

In [5]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [6]:
db_train = h5py.File("../input/datasets/cifar_rgbmean_train.hdf5")
db_test = h5py.File("../input/datasets/cifar_rgbmean_test.hdf5")

x_train_rgbmean = db_train["images"][:].astype('float32')
x_test_rgbmean = db_test["images"][:].astype('float32')

mean = np.mean(x_train_rgbmean, axis=0)
x_train_rgbmean -= mean
x_test_rgbmean -= mean

y_train_rgbmean = db_train["labels"][:]
y_test_rgbmean = db_test["labels"][:]

In [9]:
class RESNET:
    @staticmethod 
    def residual_module(data, num_filter, stride, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        shortcut = data
        
        act1 = Activation("relu")(data)
        bn1 = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(act1)
        conv1 = Conv2D(int(num_filter * 0.25), (1, 1), use_bias=False, kernel_regularizer=l2(reg))(bn1)
        
        act2 = Activation("relu")(conv1)
        bn2 = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(act2)
        conv2 = Conv2D(int(num_filter * 0.25), (3, 3), strides=stride, padding="same", use_bias=False, 
                       kernel_regularizer=l2(reg))(bn2)
        
        bn3 = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(conv2)
        conv3 = Conv2D(num_filter, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(bn3)
        
        if red:
            shortcut = Conv2D(num_filter, 1, strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)
                    
        x = add([conv3, shortcut])
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(x)
        return x
    
    @staticmethod
    def build(stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        input_tensor = Input(shape=x_train_rgbmean.shape[1:])
        
        x = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(input_tensor)
        x = Conv2D(filters[0], (3, 3), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
        
        for i in range(0, len(stages)): 
            stride = (1, 1) if i == 0 else (2, 2)
            x = RESNET.residual_module(x, filters[i + 1], stride, red=True, bnEps=bnEps, bnMom=bnMom)
            for j in range(0, stages[i] - 1): 
                x = RESNET.residual_module(x, filters[i + 1], (1, 1), bnEps=bnEps, bnMom=bnMom)
        
        x = AveragePooling2D((8, 8))(x)
        x = Flatten()(x)
        x = Dense(10, kernel_regularizer=l2(reg), kernel_constraint=maxnorm(3))(x)
        x = Activation("softmax")(x)
            
        model = Model(input_tensor, x, name="resnet")
        return model

In [27]:
model = RESNET.build((2, 2, 2), (32, 32, 64, 64), reg=0.0005)

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_148 (BatchN (None, 32, 32, 3)    12          input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 32, 32, 32)   864         batch_normalization_148[0][0]    
__________________________________________________________________________________________________
activation_112 (Activation)     (None, 32, 32, 32)   0           conv2d_121[0][0]                 
__________________________________________________________________________________________________
batch_norm

In [29]:
for (i, layer) in enumerate(model.layers): 
    print("{}\t{}".format(i, layer.__class__.__name__))

0	InputLayer
1	BatchNormalization
2	Conv2D
3	Activation
4	BatchNormalization
5	Conv2D
6	Activation
7	BatchNormalization
8	Conv2D
9	BatchNormalization
10	Conv2D
11	Conv2D
12	Add
13	Activation
14	BatchNormalization
15	Activation
16	BatchNormalization
17	Conv2D
18	Activation
19	BatchNormalization
20	Conv2D
21	BatchNormalization
22	Conv2D
23	Add
24	Activation
25	BatchNormalization
26	Activation
27	BatchNormalization
28	Conv2D
29	Activation
30	BatchNormalization
31	Conv2D
32	BatchNormalization
33	Conv2D
34	Conv2D
35	Add
36	Activation
37	BatchNormalization
38	Activation
39	BatchNormalization
40	Conv2D
41	Activation
42	BatchNormalization
43	Conv2D
44	BatchNormalization
45	Conv2D
46	Add
47	Activation
48	BatchNormalization
49	Activation
50	BatchNormalization
51	Conv2D
52	Activation
53	BatchNormalization
54	Conv2D
55	BatchNormalization
56	Conv2D
57	Conv2D
58	Add
59	Activation
60	BatchNormalization
61	Activation
62	BatchNormalization
63	Conv2D
64	Activation
65	BatchNormalization
66	Conv2D
67	Batc

In [30]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from keras.utils import plot_model
plot_model(model, to_file='models/baseline-resenet.png', show_shapes=True, show_layer_names=True)

In [31]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
#filepath=output_path + "progress/inception-weights-{epoch:02d}-{val_acc:.4f}.hdf5"
print(os.getpid())
filepath=output_path + "progress/resnet-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

2984


In [33]:
history = model.fit(x_train_rgbmean, y_train_rgbmean,
            batch_size=64,
            epochs=10,
            validation_split=0.33,
            shuffle="batch",
            callbacks=callbacks)

Train on 33500 samples, validate on 16500 samples
Epoch 1/10
33500/33500 [==============================] - 63s 2ms/step - loss: 0.3050 - acc: 0.9045 - val_loss: 0.2720 - val_acc: 0.9112
Epoch 2/10
33500/33500 [==============================] - 50s 1ms/step - loss: 0.2550 - acc: 0.9157 - val_loss: 0.2410 - val_acc: 0.9191
Epoch 3/10
33500/33500 [==============================] - 50s 1ms/step - loss: 0.2304 - acc: 0.9236 - val_loss: 0.2249 - val_acc: 0.9257
Epoch 4/10
33500/33500 [==============================] - 50s 2ms/step - loss: 0.2133 - acc: 0.9295 - val_loss: 0.2109 - val_acc: 0.9308
Epoch 5/10
33500/33500 [==============================] - 51s 2ms/step - loss: 0.2022 - acc: 0.9332 - val_loss: 0.2022 - val_acc: 0.9335
Epoch 6/10
33500/33500 [==============================] - 50s 1ms/step - loss: 0.1927 - acc: 0.9366 - val_loss: 0.2028 - val_acc: 0.9326
Epoch 7/10
33500/33500 [==============================] - 50s 1ms/step - loss: 0.1858 - acc: 0.9391 - val_loss: 0.1937 - val_acc

In [17]:
scores = model.evaluate(x_test_rgbmean, y_test_rgbmean, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 94.48%; Val: 94.16%; Test: 94.02%


In [18]:
model_json = model.to_json()
with open(output_path + "cifar_model_resnet_94.02.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(output_path + "cifar_weight_resnet_94.02.hdf5")